In [34]:
import numpy as np
import pandas as pd
import base64
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold, LeaveOneOut, LeavePOut, RepeatedStratifiedKFold, RepeatedKFold
from sklearn.preprocessing import LabelEncoder, minmax_scale, MinMaxScaler, StandardScaler
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree, export_text
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.cluster import KMeans
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, Dropout, Activation, Flatten
from sklearn.metrics import (accuracy_score,confusion_matrix, ConfusionMatrixDisplay, r2_score, silhouette_score, 
balanced_accuracy_score,cohen_kappa_score, recall_score, precision_score, f1_score)
from kneed import knee_locator, KneeLocator
import warnings
warnings.filterwarnings("ignore")

In [2]:
astro = pd.read_csv("E:/Python Programs/Astro_EuroMillions_2.csv")
astro.head()

,Date,Day,Month,Year,WeekDay,DecX,Decy,X,y
0,2/1/2024,2,1,2024,Tuesday,4901127297,7151846491012,100100100001000010100010010000001,1101000000100101011010001010000001110000100
1,5/1/2024,5,1,2024,Friday,5504049793,4718395038,101001000000100010010001010000001,100011001001111001111111010011110
2,9/1/2024,9,1,2024,Tuesday,411730306,2912394013,11000100010101000000110000010,10101101100101111001101100011101
3,12/1/2024,12,1,2024,Friday,407208514,1617184549912,11000010001011000001001000010,10111100010000111101101011110110000011000
4,16/1/2024,16,1,2024,Tuesday,6461867026,1018213345812,110000001001010000100010000010010,1110110100010010001111101010101000010100


In [3]:
astro.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Date     58 non-null     object
 1   Day      58 non-null     int64 
 2   Month    58 non-null     int64 
 3   Year     58 non-null     int64 
 4   WeekDay  58 non-null     object
 5   DecX     58 non-null     int64 
 6   Decy     58 non-null     int64 
 7   X        58 non-null     object
 8   y        58 non-null     object
dtypes: int64(5), object(4)
memory usage: 4.2+ KB


In [4]:
astro["Day"] = astro["Day"].astype(dtype='int8')
astro["Month"] = astro["Month"].astype(dtype='int8')
astro["Year"] = astro["Year"].astype(dtype='int32')

In [5]:
astro.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Date     58 non-null     object
 1   Day      58 non-null     int8  
 2   Month    58 non-null     int8  
 3   Year     58 non-null     int32 
 4   WeekDay  58 non-null     object
 5   DecX     58 non-null     int64 
 6   Decy     58 non-null     int64 
 7   X        58 non-null     object
 8   y        58 non-null     object
dtypes: int32(1), int64(2), int8(2), object(4)
memory usage: 3.2+ KB


In [ ]:
# J = astro["X"][0]
# J

In [ ]:
# K = base64.b64encode(str(J).encode())
# K

In [ ]:
# type(K)

In [41]:
X = astro[["Day", "Month", "DecX"]]
y = astro["Decy"]

In [42]:
kf1 = KFold(n_splits=5, shuffle=True)
kf1.split(X)

<generator object _BaseKFold.split at 0x00000241CA054B80>

In [43]:
for ix_train, ix_test in kf1.split(X):
    print(ix_train)
    print(ix_test)
    print()

[ 1  4  5  6  7  8  9 10 12 13 14 15 16 17 18 19 20 21 22 23 24 27 28 31
 32 33 34 35 36 37 38 39 40 41 42 43 44 45 47 49 51 52 54 55 56 57]
[ 0  2  3 11 25 26 29 30 46 48 50 53]

[ 0  2  3  4  5  6  7  8  9 10 11 13 16 17 18 19 20 21 22 23 25 26 27 29
 30 31 32 33 35 36 37 39 40 42 43 44 45 46 48 50 51 52 53 54 55 57]
[ 1 12 14 15 24 28 34 38 41 47 49 56]

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 14 15 19 21 24 25 26 28 29 30 31
 32 33 34 37 38 39 40 41 44 45 46 47 48 49 50 51 52 53 54 55 56 57]
[13 16 17 18 20 22 23 27 35 36 42 43]

[ 0  1  2  3  7 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29
 30 31 33 34 35 36 38 40 41 42 43 46 47 48 49 50 51 52 53 54 55 56 57]
[ 4  5  6  8  9 10 32 37 39 44 45]

[ 0  1  2  3  4  5  6  8  9 10 11 12 13 14 15 16 17 18 20 22 23 24 25 26
 27 28 29 30 32 34 35 36 37 38 39 41 42 43 44 45 46 47 48 49 50 53 56]
[ 7 19 21 31 33 40 51 52 54 55 57]



In [44]:
model_dt_reg = DecisionTreeRegressor(criterion='squared_error', max_depth=15)
accuracy_dt_reg = []
for ix_train, ix_test in kf1.split(X):
    X_train = X.iloc[ix_train]
    X_test = X.iloc[ix_test]
    y_train = y.iloc[ix_train]
    y_test = y.iloc[ix_test]
    model_dt_reg.fit(X_train, y_train)
    accuracy_dt_reg.append(model_dt_reg.score(X_test, y_test))
print(accuracy_dt_reg)

[-0.489390202906302, -2.0579526890241358, -3.4896139123269005, -72.21346306824908, -2.7142773943116496]


In [45]:
model_knn_reg = KNeighborsRegressor(n_neighbors=15,weights='distance')
accuracy_knn_reg = []
for ix_train, ix_test in kf1.split(X):
    X_train = X.iloc[ix_train]
    X_test = X.iloc[ix_test]
    y_train = y.iloc[ix_train]
    y_test = y.iloc[ix_test]
    model_knn_reg.fit(X_train, y_train)
    accuracy_knn_reg.append(model_knn_reg.score(X_test, y_test))
print(accuracy_knn_reg)

[-1.6680188936071088, -2.527658313944465, -0.5114721986211928, -0.021464854329838134, -0.2509028860110287]


In [46]:
loo = LeaveOneOut()
for ix_train, ix_test in loo.split(X):
    print(ix_train)
    print(ix_test)
    print()

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57]
[0]

[ 0  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57]
[1]

[ 0  1  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57]
[2]

[ 0  1  2  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57]
[3]

[ 0  1  2  3  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57]
[4]

[ 0  1  2  3  4  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33

In [47]:
loo = LeaveOneOut()
dt_model_reg = DecisionTreeRegressor(criterion='squared_error')
dt_accuracy_reg = []
for ix_train, ix_test in loo.split(X):
    X_train, X_test, y_train, y_test = X.iloc[ix_train], X.iloc[ix_test], y.iloc[ix_train], y.iloc[ix_test]
    dt_model_reg.fit(X_train, y_train)
    dt_reg_y_predict = dt_model_reg.predict(X_test)
    dt_accuracy_reg.append(dt_model_reg.score(X_test, y_test))
print(dt_accuracy_reg)
print(np.mean(dt_accuracy_reg))

[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
nan


In [48]:
rkf = RepeatedKFold()
for ix_train, ix_test in rkf.split(X):
    print(ix_train)
    print(ix_test)
    print()

[ 0  1  2  4  6  7  8 11 12 13 14 15 16 17 19 20 21 22 24 25 26 27 28 29
 31 33 34 35 36 37 38 39 40 41 42 44 45 46 47 48 49 50 51 54 55 57]
[ 3  5  9 10 18 23 30 32 43 52 53 56]

[ 0  1  2  3  4  5  6  7  8  9 10 12 13 14 15 16 17 18 21 22 23 24 25 26
 29 30 31 32 33 36 38 39 40 41 43 45 46 47 49 50 51 52 53 54 56 57]
[11 19 20 27 28 34 35 37 42 44 48 55]

[ 0  2  3  4  5  6  7  8  9 10 11 12 14 15 16 17 18 19 20 21 22 23 24 27
 28 30 32 34 35 37 38 39 40 41 42 43 44 45 46 48 52 53 54 55 56 57]
[ 1 13 25 26 29 31 33 36 47 49 50 51]

[ 1  2  3  5  8  9 10 11 12 13 15 16 18 19 20 21 22 23 24 25 26 27 28 29
 30 31 32 33 34 35 36 37 38 40 42 43 44 46 47 48 49 50 51 52 53 55 56]
[ 0  4  6  7 14 17 39 41 45 54 57]

[ 0  1  3  4  5  6  7  9 10 11 13 14 17 18 19 20 23 25 26 27 28 29 30 31
 32 33 34 35 36 37 39 41 42 43 44 45 47 48 49 50 51 52 53 54 55 56 57]
[ 2  8 12 15 16 21 22 24 38 40 46]

[ 0  1  2  3  5  7  8  9 11 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27
 29 30 31 32 33 35 36 37 38

In [49]:
model_rkf_dt_reg = DecisionTreeRegressor(criterion='squared_error', max_depth=15)
accuracy_rkf_dt_reg = []
for ix_train, ix_test in rkf.split(X):
    X_train = X.iloc[ix_train]
    X_test = X.iloc[ix_test]
    y_train = y.iloc[ix_train]
    y_test = y.iloc[ix_test]
    model_rkf_dt_reg.fit(X_train, y_train)
    accuracy_rkf_dt_reg.append(model_rkf_dt_reg.score(X_test, y_test))
print(accuracy_rkf_dt_reg)

[-0.5515152601038871, -0.7011134073813732, -0.8195992497433342, -74.63506562260555, -0.9314117537585023, -20.117584064433803, -2.6269400842038606, -0.7576331087132926, -0.19671230911435966, -7.723868474008832, -0.7832853292110686, -1.455617048978361, -2.8061337141150027, -0.3017682310745089, -6.923669606346498, -7.229257378535925, -0.36083104412649347, -14.104259036250134, -3.683422741530264, -1.4792361017885756, -0.45518199277503, -0.3887182644956544, -0.6777352786521447, -0.2677382429689479, -4.307662976649387, -6.798407541064652, -5.8696472150497865, -9.08020686516317, -0.5541232450020903, 0.2983499588550207, -3.362322418646327, -10.401227057994674, -1.0914702734090098, -0.4130564042628404, 0.023466809864292837, -0.5375746564397041, -0.549184802982529, -4.109779450321382, -0.7118989718145319, -1.4476737661889443, -3.0598738918671273, -5.7352383025549845, -0.4518247371424702, -0.4739554047090191, -18.133364030176487, -3.050018534621276, -4.399001555001717, -0.5685982463361678, -0.096

In [76]:
euro = pd.read_csv("E:/Python Programs/EuroMillions_numbers.csv")
euro.tail()

,Date,N1,N2,N3,N4,N5,E1,E2,Concat 5,Concat 2
1313,1/15/2021,4,10,27,38,40,3,11,"4,10,27,38,40","3,11"
1314,12/1/2021,2,10,18,34,35,6,9,"2,10,18,34,35","6,9"
1315,8/1/2021,18,23,37,41,42,4,6,"18,23,37,41,42","4,6"
1316,5/1/2021,12,34,37,38,42,1,3,"12,34,37,38,42","1,3"
1317,1/1/2021,16,28,32,44,48,1,9,"16,28,32,44,48","1,9"


In [77]:
for i in range(1318):
    euro["Concat 5"][i] = np.fromstring(euro["Concat 5"][i], sep=",", dtype='int32')

In [78]:
euro.tail()

,Date,N1,N2,N3,N4,N5,E1,E2,Concat 5,Concat 2
1313,1/15/2021,4,10,27,38,40,3,11,"[4, 10, 27, 38, 40]","3,11"
1314,12/1/2021,2,10,18,34,35,6,9,"[2, 10, 18, 34, 35]","6,9"
1315,8/1/2021,18,23,37,41,42,4,6,"[18, 23, 37, 41, 42]","4,6"
1316,5/1/2021,12,34,37,38,42,1,3,"[12, 34, 37, 38, 42]","1,3"
1317,1/1/2021,16,28,32,44,48,1,9,"[16, 28, 32, 44, 48]","1,9"


In [90]:
def insertion_sort(arr):
    for i in range(1, len(arr)):
        j = i
        while arr[j-1] > arr[j] and j>0:
            arr[j-1], arr[j] = arr[j], arr[j-1]
            j = j-1
    return arr

In [94]:
euro["Concat 5"] = euro["Concat 5"].apply(insertion_sort)

In [95]:
euro.head(25)

,Date,N1,N2,N3,N4,N5,E1,E2,Concat 5,Concat 2
0,12/24/2004,3,4,27,29,37,6,5,"[3, 4, 27, 29, 37]","6,5"
1,12/17/2004,15,19,22,46,49,9,2,"[15, 19, 22, 46, 49]","9,2"
2,11/26/2004,1,4,24,34,36,8,6,"[1, 4, 24, 34, 36]","8,6"
3,10/22/2004,1,9,23,25,40,3,9,"[1, 9, 23, 25, 40]","3,9"
4,1/10/2004,12,20,21,45,48,6,5,"[12, 20, 21, 45, 48]","6,5"
5,6/8/2004,5,35,44,15,24,6,5,"[5, 15, 24, 35, 44]","6,5"
6,6/18/2004,2,23,28,40,43,6,2,"[2, 23, 28, 40, 43]","6,2"
7,4/6/2004,9,13,34,41,42,7,3,"[9, 13, 34, 41, 42]","7,3"
8,5/21/2004,15,29,37,39,49,4,9,"[15, 29, 37, 39, 49]","4,9"
9,5/14/2004,1,3,21,32,39,2,6,"[1, 3, 21, 32, 39]","2,6"


In [96]:
for i in range(1318):
    euro["Concat 2"][i] = np.fromstring(euro["Concat 2"][i], sep=",", dtype='int8')

In [97]:
euro["Concat 2"] = euro["Concat 2"].apply(insertion_sort)

In [98]:
euro.head()

,Date,N1,N2,N3,N4,N5,E1,E2,Concat 5,Concat 2
0,12/24/2004,3,4,27,29,37,6,5,"[3, 4, 27, 29, 37]","[5, 6]"
1,12/17/2004,15,19,22,46,49,9,2,"[15, 19, 22, 46, 49]","[2, 9]"
2,11/26/2004,1,4,24,34,36,8,6,"[1, 4, 24, 34, 36]","[6, 8]"
3,10/22/2004,1,9,23,25,40,3,9,"[1, 9, 23, 25, 40]","[3, 9]"
4,1/10/2004,12,20,21,45,48,6,5,"[12, 20, 21, 45, 48]","[5, 6]"


In [100]:
euro.to_excel("E:/Python Programs/EuroMillions_numbers_sorted.xlsx")